In [2]:
import os
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np

def geography(left, right, bottom, top):
    #
    resolution = 12
    #
    upperindex = (90-top)*resolution
    lowerindex = (90-bottom)*resolution
    leftindex = (180+left)*resolution
    rightindex = (180+right)*resolution
    #
    rgnshape = (lowerindex-upperindex, rightindex-leftindex)
    #
    img_extent = (left, right, bottom, top)
    
    return upperindex, lowerindex, leftindex, rightindex, rgnshape, img_extent

In [ ]:
city_len = 1860
ex_flg = True

h08dir = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08'
pop_path = f'{h08dir}/global_city/dat/pop_tot_/GPW4ag__20100000.gl5'
pop = np.fromfile(pop_path, dtype='float32').reshape(2160, 4320)
riv_path = f'{h08dir}/wsi/dat/riv_out_/W5E5LR__00000000.gl5'
rivout = np.fromfile(riv_path, dtype='float32').reshape(2160, 4320)
msk_dir = f'{h08dir}/global_city/dat/vld_cty_'
cnt_dir = f'{h08dir}/global_city/dat/cty_cnt_/gpw4'
prf_dir = f'{h08dir}/global_city/dat/cty_prf_'

lonlat_path = f'{h08dir}/global_city/dat/cty_lst_/gpw4/WUP2018_300k_2010.txt'
with open(lonlat_path, "r") as input_file:
    lines = input_file.readlines()

CMY = []
world = np.zeros((2160, 4320))

for i in range(city_len):
    city_num = i+1
    msk_path = f'{msk_dir}/city_{city_num:08}.gl5'
    prf_path = f'{prf_dir}/vld_cty_/city_{city_num:08}.gl5'
    
    if not os.path.exists(prf_path):
        print(f'{i+1} is invalid prf')
        
    else:
        # population
        msk = np.fromfile(msk_path, dtype='float32').reshape(2160, 4320)
        pop_msk = np.ma.masked_where(msk != 1, pop)
        pop_sum = np.sum(pop_msk)
        print('population', city_num, pop_sum)

        # rivout
        prf = np.fromfile(prf_path, dtype='float32').reshape(2160, 4320)
        supply = np.ma.masked_where(prf != 1, rivout)
        supply_sum = np.sum(supply) * 60 * 60 * 24 * 365 / (1000) # m3/year
        print('supply', city_num, supply_sum)

        # lonlat
        line = lines[i]
        parts = line.split('\t')
        parts = [item.strip() for item in parts]
        cnt_lat = float(parts[1])
        cnt_lon = float(parts[2])
        city_name = parts[4].replace("\"", "").replace("?", "").replace("/", "")
        print('lonlat', city_num, city_name, cnt_lon, cnt_lat)

        # cmy = cubic meter(m3) per year(y)
        cmy = supply_sum/pop_sum # m3/year/person
        CMY.append([city_num, cnt_lon, cnt_lat, cmy])
        print('cmy', city_num, cmy)
        print('--------------------------------------------')
        
        if np.sum(world[msk == 1]) < 1:
            world[msk == 1] = cmy

upperindex, lowerindex, leftindex, rightindex, rgnshape, img_extent = geography(135, 145, 30, 40)
region = world[upperindex:lowerindex, leftindex:rightindex]
region_masked = np.ma.masked_where(region==0, region)
plt.imshow(region_masked)
plt.show()
    
if ex_flg is True:
    # save 1860 array
    cmy_array = np.array(CMY)
    savepath = f'{h08dir}/wsi/dat/cmy/cmy_citylonlat.npy'
    np.save(savepath, cmy_array)
    print(f'{savepath} is saved')
    
    savepath = f'{h08dir}/wsi/dat/cmy/world_citygrid.npy'
    np.save(savepath, world)
    print(f'{savepath} is saved')

population 1 31821850.0
supply 1 14240908620.0
lonlat 1 Tokyo 139.69171 35.6895
cmy 1 447.5198211291927
--------------------------------------------
population 2 22058268.0
supply 2 4420797783.75
lonlat 2 Delhi 77.21667 28.66667
cmy 2 200.41454676994584
--------------------------------------------
population 3 14540163.0
supply 3 16435625989.5
lonlat 3 Shanghai 121.45806 31.22222
cmy 3 1130.3605048650418
--------------------------------------------
population 4 20178100.0
supply 4 300657725.859375
lonlat 4 Ciudad de México (Mexico City) -99.141869 19.427318
cmy 4 14.90020001186311
--------------------------------------------
population 5 9675000.0
supply 5 2057520001.5
lonlat 5 São Paulo -46.63611 -23.5475
cmy 5 212.66356604651162
--------------------------------------------
population 6 9448362.0
supply 6 12720310699.5
lonlat 6 Kinki M.M.A. (Osaka) 135.553823 34.675834
cmy 6 1346.2979826026988
--------------------------------------------
population 7 8649229.0
supply 7 632926103.34375